Randomness is one of the most important elements of cryptography!

In [1]:
import random

Numbers and Modular Arithmetic
=====

We are going to need sets of numbers from which we drawn not only our *messages*, *secrets*, etc., and specifically we will consider the natural numbers below a given upper bound *p*, which, for mathematical reasons, we here fix as a [prime number](https://en.wikipedia.org/wiki/Prime_number). 

For simplicity we also keep it small here, but if we had a larger one it would be easy to make a mapping back and forth between these numbers and e.g. text strings.

In [2]:
P = 11

group = [ number for number in range(0, P) ]
print("The set of numbers is %s" % group)

The set of numbers is [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


Now, to stay within this set of numbers when we do addition or multiplication, arithmetic works a bit differently here.

Specifically, whenever we do an operation using our numbers, we have to somehow map it back to the set.

In [3]:
a = 5
b = 6

print(" Normal addition rules gives %d + %d = %d" % (a, b, (a + b)))
print("Modular addition rules gives %d + %d = %d" % (a, b, (a + b) % P))

 Normal addition rules gives 5 + 6 = 11
Modular addition rules gives 5 + 6 = 0


In [4]:
a = 5
b = 9

print(" Normal multiplication rules gives %d * %d = %d" % (a, b, (a * b)))
print("Modular multiplication rules gives %d * %d = %d" % (a, b, (a * b) % P))

 Normal multiplication rules gives 5 * 9 = 45
Modular multiplication rules gives 5 * 9 = 1


So while the *inverse* of 5 is 1/5 using the normal rules, when we use modular rules the inverse of 5 is 9 (when *p* = 11)!

Symmetric Encryption: One-Time Pad
=====

As it turns out, the above knowledge is everything we need to make our first cryptographic scheme that will solve our first application: securely sending a message from a *sender* to a *receiver*! 

In [5]:
OTP_MESSAGES =   [ number for number in range(0, P) ]
OTP_RANDOMNESS = [ number for number in range(0, P) ]

print(" Messages are from %s" % OTP_MESSAGES)
print("Randomness is from %s" % OTP_RANDOMNESS)

 Messages are from [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Randomness is from [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


The idea is that we can mask the message by adding an independent random component *r* to it. And since it's a symmetric encryption scheme we assume that both parties know this randomness in advance -- perhaps its a value they exchanged when they last saw each other. 

This is known as the [one-time pad](https://en.wikipedia.org/wiki/One-time_pad).

In [6]:
def otp_encrypt(m, r):
    return (m + r) % P

m = 5
r = random.choice(OTP_RANDOMNESS)
c = otp_encrypt(m, r)
print("The encryption of %d is %d" % (m, c))

def otp_decrypt(c, r):
    return (c - r) % P

n = otp_decrypt(c, r)
assert(n == m)
print("The decryption of %d is %d" % (c, n))

The encryption of 5 is 3
The decryption of 3 is 5


This encryption is perfect: without knowing the randomness, the ciphertext reveals *nothing* about the message! 

We can formalise this by saying that, after seeing the ciphertext, any guess an outsider may make about which message was encrypted can be explain by some randomness.

In [7]:
def otp_explain_one(c, m):
    return [ r for r in OTP_RANDOMNESS if otp_encrypt(m, r) == c ]

def explain_one(explainer, c):
    for guess in OTP_MESSAGES:
        explanations = otp_explain_one(c, guess)
        if explanations:
            print("Message %d can be explained by %s" % (guess, explanations))
        else:
            print("Message %d cannot be explained" % guess)
            
explain_one(otp_explain_one, c)

Message 0 can be explained by [3]
Message 1 can be explained by [2]
Message 2 can be explained by [1]
Message 3 can be explained by [0]
Message 4 can be explained by [10]
Message 5 can be explained by [9]
Message 6 can be explained by [8]
Message 7 can be explained by [7]
Message 8 can be explained by [6]
Message 9 can be explained by [5]
Message 10 can be explained by [4]


Encrypted Addition
-----

The scheme also has some homomorphic properties, meaning we can compute on encrypted values (i.e. without decrypting). 

This is because `(m1 + r1) + (m2 + r2) = (m1 + m2) + (r1 + r2)`.

In [8]:
m1 = 1
r1 = random.choice(OTP_RANDOMNESS)
c1 = otp_encrypt(m1, r1)

m2 = 2
r2 = random.choice(OTP_RANDOMNESS)
c2 = otp_encrypt(m2, r2)

def otp_add_ciphertexts(c1, c2):
    return (c1 + c2) % P

def otp_add_randomness(r1, r2):
    return (r1 + r2) % P

c3 = otp_add_ciphertexts(c1, c2)
r3 = otp_add_randomness(r1, r2)
m3 = otp_decrypt(c3, r3)
assert(m3 == m1 + m2)
print("The decryption of %d is %d" % (c3, m3))

The decryption of 8 is 3


Randomness Reuse
-----

Above we encrypted every message with a new randomness, which both sender and receiver knew. 

This means that for some applications we need to have a significant amount of pre-shared data between the two parties, which might be highly impractical -- although it is said that [the red line between the US and Russian](https://en.wikipedia.org/wiki/Moscow%E2%80%93Washington_hotline#Technology) worked like this.

As a result, one may wonder if we really have to use a new randomness every time.

In [9]:
def otp_explain_two(c1, m1, c2, m2):
    return [ (r1, r2) for r1 in OTP_RANDOMNESS for r2 in OTP_RANDOMNESS if otp_encrypt(m1, r1) == c1 and otp_encrypt(m2, r2) == c2 ]

def explain_two(explainer, c1, c2):
    for guess1 in OTP_MESSAGES:
        for guess2 in OTP_MESSAGES:
            explanations = explainer(c1, guess1, c2, guess2)
            if explanations:
                print("Messages %d,%d can be explained by %s" % (guess1, guess2, explanations))
            else:
                print("Messages %d,%d cannot be explained" % (guess1, guess2))

We first test what happens if we use two independent keys.

In [10]:
m1 = 1
r1 = random.choice(OTP_RANDOMNESS)
c1 = otp_encrypt(m1, r1)

m2 = 2
r2 = random.choice(OTP_RANDOMNESS)
c2 = otp_encrypt(m2, r2)
            
explainer = otp_explain_two
explain_two(explainer, c1, c2)

Messages 0,0 can be explained by [(6, 7)]
Messages 0,1 can be explained by [(6, 6)]
Messages 0,2 can be explained by [(6, 5)]
Messages 0,3 can be explained by [(6, 4)]
Messages 0,4 can be explained by [(6, 3)]
Messages 0,5 can be explained by [(6, 2)]
Messages 0,6 can be explained by [(6, 1)]
Messages 0,7 can be explained by [(6, 0)]
Messages 0,8 can be explained by [(6, 10)]
Messages 0,9 can be explained by [(6, 9)]
Messages 0,10 can be explained by [(6, 8)]
Messages 1,0 can be explained by [(5, 7)]
Messages 1,1 can be explained by [(5, 6)]
Messages 1,2 can be explained by [(5, 5)]
Messages 1,3 can be explained by [(5, 4)]
Messages 1,4 can be explained by [(5, 3)]
Messages 1,5 can be explained by [(5, 2)]
Messages 1,6 can be explained by [(5, 1)]
Messages 1,7 can be explained by [(5, 0)]
Messages 1,8 can be explained by [(5, 10)]
Messages 1,9 can be explained by [(5, 9)]
Messages 1,10 can be explained by [(5, 8)]
Messages 2,0 can be explained by [(4, 7)]
Messages 2,1 can be explained 

As expected, everything remains secure and every pair of messages can be explained.

Next we test what happens if we instead use the same randomness twice.

In [11]:
r = random.choice(OTP_RANDOMNESS)

m1 = 1
r1 = r
c1 = otp_encrypt(m1, r1)

m2 = 2
r2 = r
c2 = otp_encrypt(m2, r2)

explainer = lambda *x: [ rs for rs in otp_explain_two(*x) if rs[0] == rs[1] ]
explain_two(explainer, c1, c2)

Messages 0,0 cannot be explained
Messages 0,1 can be explained by [(6, 6)]
Messages 0,2 cannot be explained
Messages 0,3 cannot be explained
Messages 0,4 cannot be explained
Messages 0,5 cannot be explained
Messages 0,6 cannot be explained
Messages 0,7 cannot be explained
Messages 0,8 cannot be explained
Messages 0,9 cannot be explained
Messages 0,10 cannot be explained
Messages 1,0 cannot be explained
Messages 1,1 cannot be explained
Messages 1,2 can be explained by [(5, 5)]
Messages 1,3 cannot be explained
Messages 1,4 cannot be explained
Messages 1,5 cannot be explained
Messages 1,6 cannot be explained
Messages 1,7 cannot be explained
Messages 1,8 cannot be explained
Messages 1,9 cannot be explained
Messages 1,10 cannot be explained
Messages 2,0 cannot be explained
Messages 2,1 cannot be explained
Messages 2,2 cannot be explained
Messages 2,3 can be explained by [(4, 4)]
Messages 2,4 cannot be explained
Messages 2,5 cannot be explained
Messages 2,6 cannot be explained
Messages 2,7 c

Here we see that knowing one of the messages tells us everything about the other! Formally, we see that most message pairs can no longer be explained.

We conclude that in this scheme randomness can never be reused, and hence we are stuck with the problem of the two parties having a large amount of pre-shared data.

Discrete Logarithm and Decisional Diffie-Hellman
=====

Our next scheme can not only deal with the limitations of randomness reuse we saw above, it also has some other interesting properties. 

However, we have to switch from the perfect security we got above, where no amount of computation could ever break the scheme, to security only against "efficient" adversaries. Concretely, we introduce what is known as a [hardness assumption](https://en.wikipedia.org/wiki/Computational_hardness_assumption) which assumes that there are some mathematical problems that will take too long to solve, even on the most powerful of computers.

For this we need a bit more number theory. Specifically, we need to talk about a [generator](https://en.wikipedia.org/wiki/Generating_set_of_a_group) *g* and the fact that we can express the numbers in our domain in more than one way: like "raw" numbers as above, and as *exponents of a generator* as done here.

In [12]:
G = 6

exponents = [ number for number in range(2*P) ]
subgroup =  [ G**x % P for x in exponents ]
print(" The exponents are %s" % exponents)
print("so raising G gives %s" % subgroup)
print(" - which sorted is %s" % sorted(subgroup))

 The exponents are [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
so raising G gives [1, 6, 3, 7, 9, 10, 5, 8, 4, 2, 1, 6, 3, 7, 9, 10, 5, 8, 4, 2, 1, 6]
 - which sorted is [1, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10]


We see that `g**0 = g**(p-1)`, `g**1 = g**p`, etc. is repeating forever. Hence, we only need to consider the exponents from *0* to *q = p-1*. This gives us a new subgroup consisting of everything from the previous group, except 0.

In [13]:
Q = P - 1

exponents = [ number for number in range(Q) ]
subgroup =  [ G**x % P for x in exponents ]
print(" The exponents are %s" % exponents)
print("so raising G gives %s" % subgroup)
print(" - which sorted is %s" % sorted(subgroup))

 The exponents are [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
so raising G gives [1, 6, 3, 7, 9, 10, 5, 8, 4, 2]
 - which sorted is [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


Now the point is that it's **easy** to go from `x` to `g**x` but it is assumed **hard** to go the other way from `g**x` to `x`! This is the [discrete log (DL) assumption](https://en.wikipedia.org/wiki/Discrete_logarithm) since it's saying that computing the function `x = log(y)` such that `g**x = y` is hard.

Moreover, the [decisional Diffie-Hellman (DDH) assumption](https://en.wikipedia.org/wiki/Decisional_Diffie%E2%80%93Hellman_assumption) says that such an exponentiation looks indepedent, if the exponent is independent and unknown. More formally, it is hard to tell the difference between `(g**a, (g**a)**b)` and `(g**a, g**c)` for random and independent *a*, *b*, and *c*. 

Intuitively, this means that we again have something that looks random for outsiders, which is what we need to do encryption! However, that fact that we now have a bit more structure can be used to solve the issue from the one-time pad.

I'm cheating a bit here by the way.

Symmetric Encryption: ElGamal
=====

The [ElGamal scheme](https://en.wikipedia.org/wiki/ElGamal_encryption) is doing exactly this. 

The idea is that the sender now simply pick a fresh a randomness on his own -- i.e. that the receiver *doesn't* know -- and sends it along together with the ciphertext. Of course this only works if the randomness is somehow protected against outsiders.

As such, a secret key known by only the sender and the receiver is used to protect the randomness. But unlike the case for the one-time pad, this key can be reused several times!

In [14]:
SYMELGAMAL_MESSAGES =   [ G**exponent % P for exponent in range(Q) ]
SYMELGAMAL_RANDOMNESS = [ number for number in range(Q) ]
SYMELGAMAL_KEYS =       [ number for number in range(Q) ]

print(" Messages are from %s" % sorted(SYMELGAMAL_MESSAGES))
print("Randomness is from %s" % SYMELGAMAL_RANDOMNESS)
print("     Keys are from %s" % SYMELGAMAL_KEYS)

 Messages are from [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Randomness is from [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
     Keys are from [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


Specifically, we assume that the sender and the receiver knows a key *k*. The sender will then pick a randomness *r*, protect it by computing `g**r`, and mask the message using `(g**r)**k`. 

Using the DL assumption it is hard to extract *r* from `g**r`, and using the DDH assumption it is hard to tell the difference between `(g**r)**k` and something completely random. 

Hence, under these assumptions we essentially have a "one-time" pad where the sender can pick the randomness, and send it protected along with the ciphertext.

To decrypt, the receiver simply uses his knowledge of *k* to compute an unmask value `(g**r)**(-k)`, which gives `(g**r)**(-k) * (g**r)**k * m` that may be rewritten to `g**(-rk) * g**(rk) * m`, then `g**(-rk + rk) * m`, then `g**0 * m`, and finally `1 * m = m`.

In [15]:
def symelgamal_encrypt(m, r, k):
    onetimekey = G**r % P
    mask = onetimekey**k % P
    masked_message = (mask * m) % P
    return (onetimekey, masked_message)

k = random.choice(SYMELGAMAL_KEYS)
print("The secret key is %d" % k)

m = 5
r = random.choice(SYMELGAMAL_RANDOMNESS)
c = symelgamal_encrypt(m, r, k)
print("The encryption of %d is %s" % (m, c))

def symelgamal_decrypt(c, k):
    onetimekey, masked_message = c
    inverse_mask = onetimekey**(Q - k) % P
    return (masked_message * inverse_mask) % P

n = symelgamal_decrypt(c, k)
assert(n == m)
print("The decryption of %s is %d" % (c, n))

The secret key is 8
The encryption of 5 is (8, 3)
The decryption of (8, 3) is 5


Encrypted Multiplication
-----

This scheme also has an homomorphic property, but now allowing multiplications of the messages instead of addition.

In [18]:
k = random.choice(SYMELGAMAL_KEYS)
print("The secret key is %d" % k)

m1 = 2
r1 = random.choice(SYMELGAMAL_RANDOMNESS)
c1 = symelgamal_encrypt(m1, r1, k)
print("The encryption of %d is %s" % (m1, c1))

m2 = 3
r2 = random.choice(SYMELGAMAL_RANDOMNESS)
c2 = symelgamal_encrypt(m2, r2, k)
print("The encryption of %d is %s" % (m2, c2))

def symelgamal_mul(c1, c2):
    c1_mask, c1_message = c1
    c2_mask, c2_message = c2
    return ((c1_mask * c2_mask) % P, (c1_message * c2_message) % P)

c3 = symelgamal_mul(c1, c2)
m3 = symelgamal_decrypt(c3, k)
assert(m3 == m1 * m2)
print("The decryption of %s is %d" % (c3, m3))

The secret key is 3
The encryption of 2 is (7, 4)
The encryption of 3 is (3, 4)
The decryption of (10, 5) is 6


Asymmetric Encryption: The Real ElGamal
=====

It's a basic fact that `(g**a)**b` is exactly the same number as `(g**b)**a` -- they are both `g**(ab)`!

So note that in `symelgamal_encrypt`, when the sender computes the mask as `(g**r)**k` he could in fact just as well have computed this as `(g**k)**r`. But, this change means that we might as well simply give him `g**k` instead of asking him to compute it, in which case he doesn't need to know *k*. 

And since the DDH assumption actually says that it is hard to tell the difference between `(g**a, g**b, (g**a)**b)` and `(g**a, g**b, g**c)` then we can extend ElGamal to an asymmetric encryption scheme!

For this, the receiver first picks a **private key** *k* and then computes a **public key** as `h = g**k`. When the sender wants to send a message he uses *h* as explained above, and by the DL assumption no-one can extract *k* from *h*, and by the DDH assumption the mask when computed like this looks completely random.

As for application, we still have a sender that wishes to send a message to a receiver, but now they no longer need to have a shared secret key! They just need that the receiver has published a public key to an online "phone book" (or [PKI](https://en.wikipedia.org/wiki/Public_key_infrastructure)).

In [19]:
ASYMELGAMAL_MESSAGES =   [ G**exponent % P for exponent in range(Q) ]
ASYMELGAMAL_RANDOMNESS = [ number for number in range(Q) ]
ASYMELGAMAL_KEYS =       [ number for number in range(Q) ]

print(" Messages are from %s" % sorted(ASYMELGAMAL_MESSAGES))
print("Randomness is from %s" % ASYMELGAMAL_RANDOMNESS)
print("     Keys are from %s" % ASYMELGAMAL_KEYS)

 Messages are from [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Randomness is from [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
     Keys are from [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [20]:
k = random.choice(ASYMELGAMAL_KEYS)
h = G**k % P
print("The private key is %d" % k)
print("The public key is %d" % h)

def asymelgamal_encrypt(m, r, h):
    onetimekey = G**r % P
    mask = h**r % P
    masked_message = (mask * m) % P
    return (onetimekey, masked_message)

m = 5
r = random.choice(ASYMELGAMAL_RANDOMNESS)
c = asymelgamal_encrypt(m, r, h)
print("The encryption of %d is %s" % (m, c))

def asymelgamal_decrypt(c, k):
    onetimekey, masked_message = c
    inverse_mask = onetimekey**(Q - k) % P
    return (masked_message * inverse_mask) % P

n = asymelgamal_decrypt(c, k)
assert(n == m)
print("The decryption of %s is %d" % (c, n))

The private key is 4
The public key is 9
The encryption of 5 is (2, 3)
The decryption of (2, 3) is 5


Encrypted Multiplication
------

Nothing has changed for the ciphertexts, so we still have the homomorphic property.

In [21]:
k = random.choice(ASYMELGAMAL_KEYS)
h = G**k % P
print("The private key is %d" % k)
print("The public key is %d" % h)

m1 = 2
r1 = random.choice(ASYMELGAMAL_RANDOMNESS)
c1 = asymelgamal_encrypt(m1, r1, h)
print("The encryption of %d is %s" % (m1, c1))

m2 = 3
r2 = random.choice(ASYMELGAMAL_RANDOMNESS)
c2 = asymelgamal_encrypt(m2, r2, h)
print("The encryption of %d is %s" % (m2, c2))

def asymelgamal_mul(c1, c2):
    c1_mask, c1_message = c1
    c2_mask, c2_message = c2
    return ((c1_mask * c2_mask) % P, (c1_message * c2_message) % P)

c3 = asymelgamal_mul(c1, c2)
m3 = asymelgamal_decrypt(c3, k)
assert(m3 == m1 * m2)
print("The decryption of %s is %d" % (c3, m3))

The private key is 6
The public key is 5
The encryption of 2 is (1, 2)
The encryption of 3 is (4, 1)
The decryption of (4, 2) is 6


Key Exchange
=====

Above we considered schemes for hiding messages when sent across an insecure channel such as the internet. However, encrypting a long message using the ElGamal scheme can be very slow, and in practice a [hybrid scheme](https://en.wikipedia.org/wiki/Hybrid_cryptosystem) mixing a symmetric scheme with something else is often used.

This "something else" could in principle be an asymmetric scheme: Alice picks a fresh key for the symmetric scheme and encrypts her message using it, and then encrypts the key using Bob's public key and sends it along. This would work, but would mean that if Bob's public key is ever broken the adversary will be able to all of a sudden decrypt all messages sent to him (see [forward security](https://en.wikipedia.org/wiki/Forward_secrecy)).

In [22]:
DH_RANDOMNESS = [ number for number in range(Q) ]
DH_KEYS =       [ G**r % P for r in DH_RANDOMNESS ]

print("Randomness is from %s" % DH_RANDOMNESS)
print("     Keys are from %s" % sorted(DH_KEYS))

Randomness is from [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
     Keys are from [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


An alternative comes from the [Diffie-Hellman key exchange scheme](https://en.wikipedia.org/wiki/Diffie%E2%80%93Hellman_key_exchange) that can also be based on the DDH assumption. Here, Alice and Bob can generate a shared secret in such a way that the outsider cannot read it, and without ever sending it across the channel!

In [23]:
a = random.choice(DH_RANDOMNESS)
ga = G**a % P
print("Alice sends %d" % ga)

b = random.choice(DH_RANDOMNESS)
gb = G**b % P
print("Bob sends %d" % gb)

key_a = gb**a % P
print("Alice computes %d as her shared secret key" % key_a)

key_b = ga**b % P
print("  Bob computes %d as his shared secret key" % key_b)

assert(key_a == key_b)

Alice sends 5
Bob sends 8
Alice computes 3 as her shared secret key
  Bob computes 3 as his shared secret key


We see that the secret key is never sent in any message but just appears out of thin air!

Identification
=====

Our final application is that of proving without revealing!

One concrete application is identification, e.g. proving to a server that you are who you claim to be. Another is to prove that we know the message inside an ElGamal encryption.

Schemes for proving statements without revealing anything about the witnesses are called [zero-knowledge proof of knowledge](https://en.wikipedia.org/wiki/Zero-knowledge_proof#Discrete_log_of_a_given_value) protocols.

A simple way of doing this is as follows, again using the DL assumption.

In [27]:
ZK_SECRETS =    [ number for number in range(Q) ]
ZK_STATEMENTS = [ G**r % P for r in ZK_SECRETS ]
ZK_CHALLENGES = [ 0, 1 ]
print("   Secrets are from %s" % ZK_SECRETS)
print("Statements are from %s" % sorted(ZK_STATEMENTS))
print("Challenges are from %s" % ZK_CHALLENGES)

   Secrets are from [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Statements are from [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Challenges are from [0, 1]


We assume that a *prover* knows a secret *x* and has published a statement *y* as his public identity. 

For someone to prove that he's the rightful owner of the identity he needs to prove that he knows a matching *x* such that `y = log(x)`, which by the DL assumption is hard to do unless he already knows *x*.

In [28]:
x = random.choice(ZK_SECRETS)
y = G**x % P
print("Prover announces %d as his public identity" % y)

Prover announces 8 as his public identity


To later prove that he knows *x*, they both execute the following protocol a number of times; if the verifier ever rejects, the overall decision is to reject.

In [29]:
for i in range(10):
    
    print("Round %d" % i)
    
    r = random.choice(ZK_SECRETS)
    t = G**r % P
    print("  Prover sends %d to the verifier as a commitment" % t)

    c = random.choice(ZK_CHALLENGES)
    print("  Verifier sends %d to the prover as a challenge" % c)

    s = (r + c*x) % Q
    print("  Prover sends %d to the verifier as an answer" % s)

    b = (G**s) % P == t*(y**c) % P 
    if b:
        print("  Verifier accepts")
    else:
        print("  Verifier rejects")
        assert(False)

Round 0
  Prover sends 7 to the verifier as a commitment
  Verifier sends 0 to the prover as a challenge
  Prover sends 3 to the verifier as an answer
  Verifier accepts
Round 1
  Prover sends 5 to the verifier as a commitment
  Verifier sends 1 to the prover as a challenge
  Prover sends 3 to the verifier as an answer
  Verifier accepts
Round 2
  Prover sends 8 to the verifier as a commitment
  Verifier sends 0 to the prover as a challenge
  Prover sends 7 to the verifier as an answer
  Verifier accepts
Round 3
  Prover sends 5 to the verifier as a commitment
  Verifier sends 0 to the prover as a challenge
  Prover sends 6 to the verifier as an answer
  Verifier accepts
Round 4
  Prover sends 3 to the verifier as a commitment
  Verifier sends 0 to the prover as a challenge
  Prover sends 2 to the verifier as an answer
  Verifier accepts
Round 5
  Prover sends 1 to the verifier as a commitment
  Verifier sends 0 to the prover as a challenge
  Prover sends 0 to the verifier as an answer

Note that any prover who can answer correctly to both *c = 0* and *c = 1* must know a correct *x*:
- an accepting *c=0* execution means a *s0* such that `g**s0 == t`
- an accepting *c=1* execution means a *s1* such that `g**s1 == t * y`

Dividing the latter by the former we get `g**(s1-s0) == y` so `x = s1-s0` is a correct discrete log for *y*.